In [ ]:
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
# Local modules
sys.path.append('..')
import reproducibility
from utils import print_dict_summary
from data_funcs import load_and_fix_data, rmse
from abc import ABC, abstractmethod
from tensorflow.keras.callbacks import Callback
from utils import hash2

In [ ]:
reproducibility_file='data/reproducibility_dict.pickle'

repro={}
repro.update(load_and_fix_data(reproducibility_file))
print_dict_summary(repro)

case = 'case11'
case_data=repro[case]

In [ ]:

class ResetStatesCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        self.model.reset_states()
class RNNModel(ABC):
    def __init__(self, params: dict):
        self.params = params
        if type(self) is RNNModel:
            raise TypeError("MLModel is an abstract class and cannot be instantiated directly")
        super().__init__()

    @abstractmethod
    def fit(self, X_train, y_train, weights=None):
        pass

    @abstractmethod
    def predict(self, X):
        pass

    def eval(self, X_test, y_test):
        preds = self.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        print(f"Test RMSE: {rmse}")
        return rmse

class RNN(RNNModel):
    def __init__(self, params, loss='mean_squared_error'):
        super().__init__(params)
        self.model = self._build_model()
        # self.compile_model()

    def _build_model(self, return_sequences=False):
        inputs = tf.keras.Input(batch_shape=self.params['batch_shape'])
        x = inputs
        for i in range(self.params['rnn_layers']):
            x = tf.keras.layers.SimpleRNN(self.params['rnn_units'],activation=self.params['activation'][0],
                  stateful=self.params['stateful'],return_sequences=return_sequences)(x)
        for i in range(self.params['dense_layers']):
            x = tf.keras.layers.Dense(self.params['dense_units'], activation=self.params['activation'][1])(x)
        model = tf.keras.Model(inputs=inputs, outputs=x)
        model.compile(loss='mean_squared_error', optimizer='adam')
        if self.params["verbose_weights"]:
            print(f"Initial Weights Hash: {hash2(model.get_weights())}")
        
        return model
    # def compile_model(self):
    #     optimizer=tf.keras.optimizers.Adam(learning_rate=self.params['learning_rate'])
    #     self.model.compile(optimizer=optimizer,
    #                        loss='mean_squared_error',
    #                        metrics=self.params.get('metrics', ['accuracy']))
    def fit(self, X_train, y_train, plot=True, weights=None, callbacks=[], verbose_fit=None):
        # verbose_fit argument is for printing out update after each epoch, which gets very long
        # These print statements at the top could be turned off with a verbose argument, but then
        # there would be a bunch of different verbose params
        print(f"Training simple RNN with params: {self.params}")
        print(f"X_train hash: {hash2(X_train)}")
        print(f"y_train hash: {hash2(y_train)}")
        # Note: we overload the params here so that verbose_fit can be easily turned on/off at the .fit call 
        if verbose_fit is None:
            verbose_fit = self.params['verbose_fit']
        history = self.model.fit(
            X_train, y_train, 
            epochs=self.params['epochs'], batch_size=self.params['batch_size'],callbacks = callbacks,
            verbose=verbose_fit)
        if plot:
            self.plot_history(history)
        if self.params["verbose_weights"]:
            print(f"Fitted Weights Hash: {hash2(self.model.get_weights())}")
    
    def predict(self, X_test):
        print("Predicting with simple RNN")
        return self.model.predict(X_test)


    def plot_history(self, history):
        plt.semilogy(history.history['loss'], label='Training loss')
        if 'val_loss' in history.history:
            plt.semilogy(history.history['val_loss'], label='Validation loss')
        plt.title(case + ' Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(loc='upper left')
        plt.show()


In [ ]:
import yaml

with open("params.yaml") as file:
    params = yaml.safe_load(file)["rnn_stateless"]


In [ ]:
N = len(repro[case]["fm"]) # total observations
# train_ind = int(N*.8) # index of train/val split
train_ind = 300
# test_ind = int(N*.9) # index of train/test split
# test_ind = 500

X = np.vstack((repro[case]["Ed"], repro[case]["Ew"])).T
y = repro[case]["fm"]

X_train = X[:train_ind]
# X_val = X[train_ind:test_ind]
X_test = X[train_ind:]
y_train = y[:train_ind].reshape(-1,1)
# y_val = y[train_ind:test_ind].reshape(-1,1)
y_test = y[train_ind:].reshape(-1,1)

print(f"Total Observations: {N}")
print(f"Num Training: {X_train.shape[0]}")
# print(f"Num Validation: {X_val.shape[0]}")
print(f"Num Test: {X_test.shape[0]}")

In [ ]:
from moisture_rnn import staircase
X_train, y_train = staircase(X_train, y_train, timesteps = params["timesteps"], datapoints = len(y_train), verbose=True)
print("~"*50)
# X_val, y_val = staircase(X_val, y_val, timesteps = params["timesteps"], datapoints = len(y_val))
# print("~"*50)
X_test, y_test = staircase(X_test, y_test, timesteps = params["timesteps"], datapoints = len(y_test), verbose=True)

In [ ]:
samples, timesteps, features = X_train.shape
batch_size = samples # Single batch for testing

params.update({
    'batch_shape': (batch_size,timesteps,features),
    'batch_size': batch_size # Single Batch for testing
})
params.update({'epochs': 5000})
params.update({'stateful': True})

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)

In [ ]:
rnn.fit(X_train, y_train, verbose_fit=False)

In [ ]:
rnn.model.summary()